In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder

import os
import re

In [57]:
col_names = ['phrase_id','sentence_id','phrase','sentiment']
train = pd.read_csv('../../dataset/MovieReviews/train.tsv',names=col_names,sep='\t',)
test = pd.read_csv('../../dataset/MovieReviews/test.tsv',names=col_names,sep='\t',)
train = train.drop(0)
test = test.drop(0)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (0,1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train.sentiment = train.sentiment.apply(lambda x:int(x))
train.sentiment.unique()

array([1, 2, 3, 4, 0])

In [ ]:
# dummies = pd.get_dummies(data.sentiment,prefix='sentiment')
# merged = pd.concat([data,dummies],axis='columns')
# data = merged.drop(['sentiment'], axis='columns')
# data.head()

In [ ]:
import string
def handle_text(text):
    text = text.lower()
    text = re.sub('-',' ',text)
    text = text.translate(str.maketrans('','', string.punctuation))
    text = re.sub('\s+',' ',text)
    text = re.sub('\b\w\b','',text)
    return text

In [ ]:
train['cleaned_text'] = train['phrase'].apply(handle_text)
test['cleaned_text'] = test['phrase'].apply(handle_text)
train.head()

In [ ]:
train = train.drop(columns=['phrase_id','sentence_id','phrase'])
 = test.drop(columns=['phrase_id','sentence_id','phrase'])
train.head()

In [ ]:
split_point = int(len(train) * 0.8)
with open('moviereview.train','w',encoding='utf8') as f:
    for row in train[:split_point].iterrows():
        f.write('{}\t__label__{}\n'.format(row[1]['cleaned_text'],row[1]['sentiment']))
with open('moviereview.test','w',encoding='utf8') as f:
    for row in train[split_point:].iterrows():
        f.write('{}\t__label__{}\n'.format(row[1]['cleaned_text'],row[1]['sentiment']))
import fasttext
model = fasttext.train_supervised(input="moviereview.train")
model.test("moviereview.test")

In [ ]:
import gensim.downloader
print(list(gensim.downloader.info()['models'].keys()))
glove_vectors = gensim.downloader.load('glove-twitter-25')

In [ ]:
train.loc[(train.cleaned_text == '') | (train.cleaned_text == ' '),'cleaned_text'] = np.nan
train = train.dropna(subset=['cleaned_text'])
train.head()

In [ ]:
def text2vec(text):
    texts = []
    for t in text.strip().split(' '):
        try:
            texts.append(glove_vectors[t])
        except Exception as e:
            # 未登录词，设置一个随机向量
            texts.append(np.random.uniform(-1, 1, 25))
    texts = np.array(texts)
    return np.sum(texts,axis=0)/len(texts[0])
#train['train_text_vec'] = train.cleaned_text.apply(text2vec)

In [ ]:
train.head()

In [ ]:
dtrain = xgb.DMatrix(data=np.array(train['train_text_vec'].tolist()),label=np.array(train['sentiment'].tolist()))

In [ ]:
params = {
    'booster': 'gbtree',
    'objective': 'multi:softmax', #多分类
    'num_class': 5, #类别数
    'gamma': 0.1,
    'max_depth': 6,
    'lambda': 2, # L2正则化系数
    'subsample': 0.7, # 生成树算法采用的列采样大小
    'colsample_bytree': 0.7,
    'min_child_weight': 3,
    'silent': 0,
    'eta': 0.1, # shrinkage
    'seed': 1000,
    'nthread': 4, # cpu线程
}

In [ ]:
num_rounds = 500
model = xgb.train(params, dtrain, num_rounds)

In [ ]:
test['test_text_vec'] = test.cleaned_text.apply(text2vec)

In [ ]:
dtest = xgb.DMatrix(data=np.array(test['test_text_vec'].tolist()))

In [ ]:
y_pred = model.predict(dtest)

In [ ]:
submission = pd.read_csv('../../dataset/MovieReviews/sampleSubmission.csv')
submission.Sentiment = y_pred
submission.Sentiment = submission.Sentiment.apply(lambda x:int(x))

In [ ]:
submission.to_csv('submission.csv', index=False)

,phrase_id,sentence_id,phrase,sentiment_0,sentiment_1,sentiment_2,sentiment_3,sentiment_4
1,1,1,A series of escapades demonstrating the adage ...,0,1,0,0,0
2,2,1,A series of escapades demonstrating the adage ...,0,0,1,0,0
3,3,1,A series,0,0,1,0,0
4,4,1,A,0,0,1,0,0
5,5,1,series,0,0,1,0,0


In [4]:
import string
def handle_text(text):
    text = text.lower()
    text = re.sub('-',' ',text)
    text = text.translate(str.maketrans('','', string.punctuation))
    text = re.sub('\s+',' ',text)
    text = re.sub('\b\w\b','',text)
    return text

In [58]:
train['cleaned_text'] = train['phrase'].apply(handle_text)
test['cleaned_text'] = test['phrase'].apply(handle_text)
train.head()

,phrase_id,sentence_id,phrase,sentiment,cleaned_text
1,1,1,A series of escapades demonstrating the adage ...,1,a series of escapades demonstrating the adage ...
2,2,1,A series of escapades demonstrating the adage ...,2,a series of escapades demonstrating the adage ...
3,3,1,A series,2,a series
4,4,1,A,2,a
5,5,1,series,2,series


In [12]:
train = train.drop(columns=['phrase_id','sentence_id','phrase'])
 = test.drop(columns=['phrase_id','sentence_id','phrase'])
train.head()

,sentiment,cleaned_text
1,1,a series of escapades demonstrating the adage ...
2,2,a series of escapades demonstrating the adage ...
3,2,a series
4,2,a
5,2,series


In [86]:
split_point = int(len(train) * 0.8)
with open('moviereview.train','w',encoding='utf8') as f:
    for row in train[:split_point].iterrows():
        f.write('{}\t__label__{}\n'.format(row[1]['cleaned_text'],row[1]['sentiment']))
with open('moviereview.test','w',encoding='utf8') as f:
    for row in train[split_point:].iterrows():
        f.write('{}\t__label__{}\n'.format(row[1]['cleaned_text'],row[1]['sentiment']))
import fasttext
model = fasttext.train_supervised(input="moviereview.train")
model.test("moviereview.test")

In [13]:
import gensim.downloader
print(list(gensim.downloader.info()['models'].keys()))
glove_vectors = gensim.downloader.load('glove-twitter-25')

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [30]:
train.loc[(train.cleaned_text == '') | (train.cleaned_text == ' '),'cleaned_text'] = np.nan
train = train.dropna(subset=['cleaned_text'])
train.head()

,sentiment,cleaned_text
1,1,a series of escapades demonstrating the adage ...
2,2,a series of escapades demonstrating the adage ...
3,2,a series
4,2,a
5,2,series


In [60]:
def text2vec(text):
    texts = []
    for t in text.strip().split(' '):
        try:
            texts.append(glove_vectors[t])
        except Exception as e:
            # 未登录词，设置一个随机向量
            texts.append(np.random.uniform(-1, 1, 25))
    texts = np.array(texts)
    return np.sum(texts,axis=0)/len(texts[0])
#train['train_text_vec'] = train.cleaned_text.apply(text2vec)

In [32]:
train.head()

,sentiment,cleaned_text,train_text_vec
1,1,a series of escapades demonstrating the adage ...,"[0.05736917, 0.11810215, -0.19433112, 0.160393..."
2,2,a series of escapades demonstrating the adage ...,"[-0.025379758, 0.0167336, -0.029060083, 0.0435..."
3,2,a series,"[0.0455864, -0.00042160033, 0.033739198, -0.00..."
4,2,a,"[0.008517601, 0.012414, 0.0070776, 0.0349992, ..."
5,2,series,"[0.0370688, -0.0128356, 0.0266616, -0.0368824,..."


In [35]:
dtrain = xgb.DMatrix(data=np.array(train['train_text_vec'].tolist()),label=np.array(train['sentiment'].tolist()))

In [41]:
params = {
    'booster': 'gbtree',
    'objective': 'multi:softmax', #多分类
    'num_class': 5, #类别数
    'gamma': 0.1,
    'max_depth': 6,
    'lambda': 2, # L2正则化系数
    'subsample': 0.7, # 生成树算法采用的列采样大小
    'colsample_bytree': 0.7,
    'min_child_weight': 3,
    'silent': 0,
    'eta': 0.1, # shrinkage
    'seed': 1000,
    'nthread': 4, # cpu线程
}

dict_items([('booster', 'gbtree'), ('objective', 'multi:softmax'), ('num_class', 5), ('gamma', 0.1), ('max_depth', 6), ('lambda', 2), ('subsample', 0.7), ('colsample_bytree', 0.7), ('min_child_weight', 3), ('silent', 0), ('eta', 0.1), ('seed', 1000), ('nthread', 4)])

In [42]:
num_rounds = 500
model = xgb.train(params, dtrain, num_rounds)

[17:17:21] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:541: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[17:17:22] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [61]:
test['test_text_vec'] = test.cleaned_text.apply(text2vec)

In [62]:
dtest = xgb.DMatrix(data=np.array(test['test_text_vec'].tolist()))

In [63]:
y_pred = model.predict(dtest)

In [68]:
submission = pd.read_csv('../../dataset/MovieReviews/sampleSubmission.csv')
submission.Sentiment = y_pred
submission.Sentiment = submission.Sentiment.apply(lambda x:int(x))

In [69]:
submission.to_csv('submission.csv', index=False)